#Problème - Session n°2 : une variable cachée

Dans ce problème, on travaille sur un jeu de données comportant 50.000 entrées $x_i$ et des cibles $y_i$. Les entrées sont des vecteurs de taille 10 (au format torch), les cibles sont des scalaires construits à partir de cinq fonctions différentes ($f_0$, ..., $f_4$) : \

$$ \forall i, \exists k\in [\![0 \;;4]\!]  \:\: \text{tel que} \: f_k(x_i) = y_i $$

Ces fonctions sont inconnues, ainsi que l'indice $k$. Par contre, on sait que le groupe des 1000 premières cibles ont été construites à partir du même indice  $k$, de même pour les mille  suivantes, et ainsi de suite.

Le but est de parvenir à rassembler les groupes de cibles qui ont été générées avec le même indice $k$ (avec la même fonction).

In [2]:
# Example d'échantillonnage du dataset
import torch
from torch.utils.data import DataLoader

! git clone https://github.com/nouhalahyen/exam_2025_session2.git
! cp exam_2025_session2/utils/utils.py .
from utils import Problem1Dataset

dataset = Problem1Dataset()
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in dataloader:
    x_batch, y_batch, k_batch, idx_batch = batch
    print("Batch input shape:", x_batch.shape)
    print("Batch target shape:", y_batch.shape)
    print("Batch k shape:", k_batch.shape) # indice k (pas utilisable à l'entraînement)
    print("Batch indices shape:", idx_batch.shape)
    break

Cloning into 'exam_2025_session2'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 89 (delta 5), reused 4 (delta 4), pack-reused 80 (from 2)
Receiving objects: 100% (89/89), 570.04 KiB | 4.35 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Batch input shape: torch.Size([32, 10])
Batch target shape: torch.Size([32, 1])
Batch k shape: torch.Size([32])
Batch indices shape: torch.Size([32])


**Consignes :**
- Entraîner l'architecture proposée dans la cellule suivante.
- Montrer que les vecteurs 2D de self.theta permettent de répondre
  au problème posé.
- Décrire le rôle de self.theta, du vector noise \
 et ainsi que la raison de la division par 1000 (**indices // 1000** dans le code).

In [4]:
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
class DeepMLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super(DeepMLP, self).__init__()
        self.theta = nn.Parameter(torch.randn(50, 2))
        self.fc1 = nn.Linear(input_dim + 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, indices):
        theta_batch = self.theta[indices // 1000, :]
        noise = torch.normal(mean=torch.zeros_like(theta_batch),
                             std=torch.ones_like(theta_batch))
        x = torch.cat([x, theta_batch + noise], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x, theta_batch

self.theta → self.theta est une matrice de taille (50, 2) qui représente des vecteurs latents associés aux groupes de 1000 échantillons.
Chaque ligne de self.theta correspond à un groupe de 1000 échantillons et sert de signature unique pour ce groupe. self.theta Ajoute une information latente pour représenter chaque groupe.
. \\

noise → Empêche la mémorisation et améliore la généralisation. (Prévient l'overfitting) \\

indices // 1000 → Associe le même vecteur theta aux groupes de 1000 échantillons pour capturer leur structure cachée. en effet, L’indice idx_batch correspond aux indices globaux des données (0 à 49 999).
En faisant indices // 1000, on obtient un nombre entre 0 et 49, ce qui permet :
D’assigner un même theta aux 1000 échantillons d’un même groupe.
De garantir que chaque bloc de 1000 exemples utilise le même vecteur latent de theta, ce qui permet au modèle de détecter la structure des groupes




In [ ]:

# Initialiser le modèle, la fonction de perte et l'optimiseur
model = DeepMLP(input_dim=10, output_dim=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Définir les paramètres d'entraînement
batch_size = 64  # Vous pouvez ajuster la taille du batch
epochs = 100  # Vous pouvez augmenter le nombre d'époques

# Créer le DataLoader
dataset = Problem1Dataset()
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Entraîner le modèle
for epoch in range(epochs):
    total_loss = 0
    for batch in dataloader:
        x_batch, y_batch, _, idx_batch = batch

        optimizer.zero_grad()
        output, theta_batch = model(x_batch, idx_batch)
        loss = criterion(output.squeeze(), y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")

# Visualiser les vecteurs theta après l'entraînement
theta_values = model.theta.detach().numpy()
plt.figure(figsize=(8,6))
plt.scatter(theta_values[:, 0], theta_values[:, 1], c=range(50), cmap='viridis', s=50)
plt.colorbar(label="Group Index")
plt.xlabel("Theta Dim 1")
plt.ylabel("Theta Dim 2")
plt.title("Visualisation des Vecteurs Theta")
plt.show()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/100, Loss: 180.9774
Epoch 2/100, Loss: 174.9205
Epoch 3/100, Loss: 174.2381
Epoch 4/100, Loss: 174.1938
Epoch 5/100, Loss: 174.3409
Epoch 6/100, Loss: 173.9078
Epoch 7/100, Loss: 173.6738
Epoch 8/100, Loss: 173.6752
Epoch 9/100, Loss: 173.6289
Epoch 10/100, Loss: 173.5973
Epoch 11/100, Loss: 173.5147
Epoch 12/100, Loss: 173.5775
Epoch 13/100, Loss: 173.4692
Epoch 14/100, Loss: 173.7033
Epoch 15/100, Loss: 173.5190
Epoch 16/100, Loss: 173.5235
Epoch 17/100, Loss: 173.3754
Epoch 18/100, Loss: 173.5140
Epoch 19/100, Loss: 173.3985
Epoch 20/100, Loss: 173.4482
